### Libraries:

In [ ]:
# !pip install Janitor

In [ ]:
# !pip install pyjanitor

In [ ]:
import janitor
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

### Function for missing frame:

In [ ]:
def fill_current_missing(data):
    print(data['Frame Number'])
    return data

### Dataset with missing frame by file:

In [ ]:
csv_path = r'C:\Users\User\Desktop\rabotno ML\ML- Final Project\Brainster data\1f - Data set'

csv_files = glob.glob(f'{csv_path}/*.csv')

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    current_missing = df[(df['Frame Number'].diff() > 1) & (df['Stimuli'].diff() == 0)]
    if len(current_missing) > 0:
        print("DATA IS MISSING:", file, len(current_missing))
        new_rows = fill_current_missing(current_missing)
        df = pd.concat([df, new_rows])
    dfs.append(df)

combined = pd.concat(dfs)

### Dataset with missing frame total:

In [ ]:
csv_path = r'C:\Users\User\Desktop\rabotno ML\ML- Final Project\Brainster data\1f - Data set'

csv_files = glob.glob(f'{csv_path}/*.csv')

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

df_all = pd.concat(dfs)

current_missing = df_all[(df_all['Frame Number'].diff() > 1) & (df_all['Stimuli'].diff() == 0)]
if len(current_missing) > 0:
    print("DATA IS MISSING:", len(current_missing))
    print(current_missing['Frame Number'])

### EDA

In [ ]:
combined.isnull().sum()

### DATA IS MISSING: 317

In [ ]:
combined.duplicated().sum()

In [ ]:
combined.shape

In [ ]:
combined.describe()

In [ ]:
combined.columns

In [ ]:
combined.dtypes

In [ ]:
combined.nunique()

In [ ]:
combined.std()

In [ ]:
num_cols = combined.select_dtypes(include=np.number).columns.tolist()
for i in num_cols:
    plt.figure(figsize=(10,10))
    y=sns.histplot(x=i, data=combined, shrink=.9,stat='count',kde = True,color='yellow',)
    y.bar_label(y.containers[0])
    plt.title('data-set_number')
    plt.xlabel(i)
    plt.show()

In [ ]:
plt.figure(figsize=(20,10))
y=sns.histplot(x='Stimuli', data=combined, bins=12, shrink=.9,stat='count',kde = True,color='red',)
y.bar_label(y.containers[0])
plt.title('Stimuli')
plt.xlabel('Count')
plt.ylabel('Stimuli-Count')
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
y= sns.barplot(data=combined, x = 'Stimuli', y='Frame Number',color = 'orange')
y.bar_label(y.containers[0])
plt.title('Stimuli vs Frame Number')
plt.show()

In [ ]:
plt.figure(figsize=(50,50))
c= combined.corr().round(2)
# sns.heatmap(c,cmap="BrBG",annot=True)
c

In [ ]:
stim = combined['Stimuli'].value_counts().sort_values(ascending=True, key=lambda x: x.index)
stim

In [ ]:
frame =combined['Frame Number'].value_counts().sort_values(ascending=True, key=lambda x: x.index)
frame

In [ ]:
column_name = 'Frame Number'

min_value = combined[column_name].min()
max_value = combined[column_name].max()

print("Minimum value in column '{}': {}".format(column_name, min_value))
print("Maximum value in column '{}': {}".format(column_name, max_value))

### Code for NaN values for all file:

In [ ]:
csv_path = r'C:\Users\User\Desktop\rabotno ML\ML- Final Project\Brainster data\1f - Data set'

csv_files = glob.glob(f'{csv_path}/*.csv')

dfs = []
for file in tqdm(csv_files, total = len(csv_files)):
    df = pd.read_csv(file)
    for g, c_df in df.groupby('Stimuli'):
        а=c_df.complete('Study_Folder','Subject_Folder','Stimuli',{"Frame Number": range(0, c_df['Frame Number'].max())},sort=True)

        df = pd.concat([а])
        dfs.append(а)

combined = pd.concat(dfs).reset_index(drop=True)

In [ ]:
combined

### Show NaN values:

In [ ]:
combined.iloc[3380:3400]

In [ ]:
combined.isnull().sum()

### Save csv file data with NaN:

In [ ]:
combined.to_csv('data_with NaN_1.csv', index=False)

In [ ]:
from google.colab import files

files.download('data_with NaN_1.csv')

### Read dataset with NaN:

In [ ]:
data= pd.read_csv('data_with NaN_1.csv')
data

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import pandas as pd
# import io

# df = pd.read_csv(io.BytesIO(uploaded['data_with NaN_1.csv']))
# df

### Drop columns:

In [ ]:
columns_save = ['Study_Folder', 'Subject_Folder', 'Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H', 'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle', 'X_Predictions', 'Y_Predictions']
data.drop(columns=[col for col in data.columns if col not in columns_save], inplace=True)

In [ ]:
data

In [ ]:
data.describe()

### Data1_withoutNaN - Filled:

In [ ]:
dfs=[]
for g, c_df in data.groupby('Stimuli'):
    if c_df.isna().sum().sum() > 0:
        for c in c_df.columns[4:]:
            #print(g,c)
            na_indexes = c_df[c].isna()
            #print(c_df[na_indexes][c])

            c_df[c] = c_df[c].interpolate()

    dfs.append(c_df)

data1_withoutNan = pd.concat(dfs).reset_index(drop=True)

In [ ]:
data1_withoutNan.isna().sum()

### Save csv file data without NaN:

In [ ]:
data1_withoutNan.to_csv('data1_filled.csv', index=False)

In [ ]:
# from google.colab import files

# files.download('data1_filled.csv')